# `pandarallel` with Controlled Number of Progress Bars

Up until at least version 1.6.4 of `pandarallel`, it displayed 1 progress bar from every 1 worker process.  With a sufficiently large number of workers, this becomes overwhelming.

This notebook demos a modification to `pandarallel` which exposes control over how many progress bars should be displayed and maps each worker process to one and only one of those progress bars.  In a multi-node `dragon` execution configuration (which is _not_ demonstrated here), some nodes may be slower/faster than others and it may be helpful to see the relative progress/speed of one cluster's nodes versus others -- this motivates showing more than just a single progress bar representing all workers.

In [1]:
!pip install pandarallel

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
import dragon
import multiprocessing

import cloudpickle

import numpy as np
import pandas as pd
import time

import pandarallel; pandarallel.__version__

'1.6.5'

In [3]:
multiprocessing.set_start_method("dragon")
pandarallel.core.dill = cloudpickle
ctx = multiprocessing.get_context("dragon")
ctx.Manager = type("PMgr", (), {"Queue": ctx.Queue})
pandarallel.core.CONTEXT = ctx
pandarallel.pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 128 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [4]:
num_rows = 10

df = pd.DataFrame(
    {
        "seqnum": np.arange(42, (42 + num_rows), dtype=int),
        #"metric_A": np.random.rand(num_rows),
        #"metric_B": np.random.rand(num_rows),
        "metric_C": np.random.rand(num_rows),
        "alt_seq": np.random.randint(low=42, high=(42 + num_rows), size=(num_rows,)),
        "label": np.array(list("ATCG"))[np.random.randint(0, 4, num_rows)],
    },
)

In [5]:
df.head()

,seqnum,metric_C,alt_seq,label
0,42,0.090634,50,C
1,43,0.534844,45,C
2,44,0.426885,50,T
3,45,0.361246,47,T
4,46,0.381398,45,C


The use of a global variable inside a lambda function demonstrates key functionality from `cloudpickle` that is not otherwise available through `dill`.

In [6]:
cutoff = 0.3

Running this next cell will cause as many progress bars to be displayed as there are workers (potentially a lot).

In [17]:
start = time.monotonic()
df['highlow_C'] = df['metric_C'].parallel_apply(lambda x: x < cutoff)
stop = time.monotonic()
tot_time = stop - start
time_dict = {}
time_dict["1"] = tot_time

Now we have our new column of values in our `pandas.DataFrame`.

In [8]:
df.head()

,seqnum,metric_C,alt_seq,label,highlow_C
0,42,0.090634,50,C,True
1,43,0.534844,45,C,False
2,44,0.426885,50,T,False
3,45,0.361246,47,T,False
4,46,0.381398,45,C,False


We can change our minds about how many progress bars to display, at will.

In [9]:
pandarallel.pandarallel.initialize(progress_bar=10)  # Will display a total of 10 progress bars.

INFO: Pandarallel will run on 128 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [18]:
start = time.monotonic()
df['highlow_C'] = df['metric_C'].parallel_apply(lambda x: x < cutoff)
stop = time.monotonic()
tot_time = stop - start
time_dict["2"] = tot_time

There will be plenty of use cases / scenarios where a single progress bar is all we want.

In [11]:
pandarallel.pandarallel.initialize(progress_bar=1)  # Will display 1 progress bar representing all workers.

INFO: Pandarallel will run on 128 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [19]:
start = stop = time.monotonic()
df['highlow_C'] = df['metric_C'].parallel_apply(lambda x: x < cutoff)
stop = time.monotonic()
tot_time = stop - start
time_dict["3"] = tot_time

In [33]:
print("parallel_apply","\t", "Time (nanoseconds)")
for key, value in time_dict.items():
    print("{:<20} {:<20}".format(key, value))

parallel_apply 	 Time (nanoseconds)
1          2.7243981696665287  
2          2.416624452918768   
3          2.3884036764502525  


Though it is very minor compared to the overall wall time, reducing the number of progress bars displayed can shave off a small amount of execution time.